In [ ]:
"""
PoGaIN: Poisson-Gaussian Image Noise Modeling from Paired Samples

Authors: Nicolas Bähler, Majed El Helou, Étienne Objois, Kaan Okumuş, and Sabine
Süsstrunk, Fellow, IEEE.

This file contains utility methods.
"""

import numpy as np
from PIL import Image
from scipy.stats import norm, poisson


def load_image(path: str, keep_shape=False) -> tuple[np.ndarray, tuple[int, int]]:
    # Load the image and convert into numpy array
    img = Image.open(path)
    array = np.asarray(img)

    if len(array.shape) == 3:
        array = np.mean(array, axis=2)

    array = array / 255

    return array if keep_shape else (array.flatten(), array.shape)


def add_noise(x: np.ndarray, a: float, b: float, seed=None) -> np.ndarray:
    np.random.seed(seed)
    n = len(x)

    poisson_noise = poisson.rvs(mu=a * x, size=n)
    gaussian_noise = norm.rvs(scale=b, size=n)
    return poisson_noise / a + gaussian_noise


def create_fake_image(n, lower=0, upper=1) -> np.ndarray:
    return np.random.uniform(lower, upper, size=n)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
import imageio

def add_noise_to_image(image_path, a, b, seed):
    # Define the root directory
    root = os.path.join(os.path.abspath(''), "..")
    # Load noise-free image
    x, shape = load_image(image_path)

    # Synthesize and add noise to image
    y = add_noise(x, a, b, seed)

    y = y.reshape(shape)
    y_scaled = ((y - y.min()) / (y.max() - y.min()) * 255).astype('uint8')
    noisy_image_path = os.path.join(root,"Project", "data", "test", os.path.basename(image_path))
    imageio.imwrite(noisy_image_path, y_scaled)
    print(f"Noisy image saved at: {noisy_image_path}")

# Example usage:
add_noise_to_image("data/test_original/AAKRKIZP.png", 32, 0.1, 42)

# Show noise-free and the noisy image